<a href="https://colab.research.google.com/github/iurykischka/Proc.-de-sinais/blob/master/Ex/Ex08_FiltrosDigitais_IURY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex08 - `[seu nome] (RA)`

Filtros digitais.

**HONESTIDADE ACADÊMICA**

Todo o trabalho feito no sentido do cumprimento das expectativas deste curso deve ser exclusivamente seu. A colaboração de terceiros na realização dos trabalhos não é permitida.

Ver ou copiar o trabalho de outro indivíduo do curso ou retirar material de um livro, site ou outra fonte, mesmo em parte e apresentá-lo como seu próprio constitui desonestidade acadêmica, assim como mostrar ou dar a sua obra, mesmo em parte, a outro estudante. Da mesma forma, é desonestidade acadêmica fornecer ou tornar as soluções disponíveis dos trabalhos para os indivíduos que fazem ou poderão fazer este curso no futuro. Você está convidado a discutir o material do curso com os outros, a fim de melhor compreendê-lo. Você pode até discutir sobre os trabalhos com os colegas, mas você não pode compartilhar códigos, funções, projetos, tabelas, desenhos esquemáticos, mapas ou diagramas.

Você pode e deve recorrer à Web para obter referências na busca de soluções para os trabalhos, mas não por soluções definitivas para os problemas. No entanto, deve-se citar explicitamente no projeto a origem de qualquer procedimento ou técnica que você descubra fora do curso.

Todas as formas de desonestidade acadêmica são tratadas com rigor.


**Instruções:**
- Não apague os comentários que já existem nas células de código. Mas é claro que você pode adicionar outros comentários.
- Não inclua mais células do que o especificado e mantenha a organização.
- Use `plt.figure(figsize=(10,1))` para ajustar o tamanho das figuras e sempre coloque rótulos.

**Objetivos**
- Implementar e analisar filtros FIR digitais
- Implementar e analisar filtros IIR digitais 

## Registre o seu RA

Primeiro, registre o seu RA na variável indicada.


<mark>**Fazer:** Escreva o seu RA na variável `RA`.</mark>

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ###
RA = None
### FIM DO CÓDIGO ###
print(RA)

**Saída esperada**:
1. Seu número de RA
---

## O problema

Nesta atividade, você vai projetar e implementar filtros digitais FIR e IIR. Os filtros projetados serão usados em sinais de áudio (são os mesmo arquivos usados no Ex02).

<mark> **Faça**</mark>: execute o código abaixo para carregar os áudios.

In [ ]:
# Importa bibliotecas
import numpy as np, scipy as sp, matplotlib.pyplot as plt, IPython.display as ipd
from scipy.io import wavfile
from scipy import signal
import requests

# Som do diapasão
response = requests.get('https://github.com/ifmg-betim/2022.2_AUT.040_ProcSin/blob/main/Ex/Ex02_diapasao.wav?raw=true')
open("diapasao.wav", "wb").write(response.content)
fs, diapasao = wavfile.read('diapasao.wav')
diapasao = diapasao[:90000] # sinal do diapasão
diapasao_i = (diapasao-np.mean(diapasao))/np.std(diapasao)

response = requests.get('https://github.com/ifmg-betim/2022.2_AUT.040_ProcSin/blob/main/Ex/Ex02_flauta.wav?raw=true')
open("flauta.wav", "wb").write(response.content)
fs, flauta = wavfile.read('flauta.wav')
flauta = flauta[:90000] # sinal da flauta
flauta_i = (flauta-np.mean(flauta))/np.std(flauta)

response = requests.get('https://github.com/ifmg-betim/2022.2_AUT.040_ProcSin/blob/main/Ex/Ex02_cello.wav?raw=true')
open("cello.wav", "wb").write(response.content)
fs, cello = wavfile.read('cello.wav')
cello = cello[10000:100000,0] # sinal do violoncelo
cello_i = (cello-np.mean(cello))/np.std(cello)

# vetor tempo
tempo = np.arange(len(diapasao))/fs # variável TEMPO (em segundos)
N = len(tempo) #tamanho do sinal
n = np.arange(N) #vetor de amostras
np.random.seed(RA)
r = np.dot(np.random.uniform(.01, .05, 100), np.sin(np.random.uniform(2e3, fs/2,100).reshape((100,1))
                                          *2*np.pi*tempo+np.random.uniform(0, np.pi, (100,1))) )
diapasao = diapasao_i + r
flauta = flauta_i + r
cello = cello_i + r

## Sinais a serem analisado

O código acima criou as variáveis abaixo para serem usadas:
- `tempo`: vetor tempo apresentado na unidade "segundos", com frequência de amostragem de $f_s=44100$ Hz. Este vetor de tempo pode ser usado para os três sinais descritos abaixo.
- `diapasao`: vetor com amplitude do sinal de áudio do diapasão.
- `flauta`: vetor com amplitude do sinal de áudio da flauta.
- `violoncelo`: vetor com amplitude do sinal de áudio do violoncelo.

O código abaixo já implementa cada um dos sinais no domínio do tempo e um player para execução do áudio. Falta fazer a análise dos sinais no domínio da frequência.

<mark> **Faça**</mark>: plote a FFT dos sinais abaixo, adicionando código nos espaços indicados. Cole na primeira célula a função criada para calcular a FFT do sinal.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### (função que implementa a FFT)
None
### FIM DO CÓDIGO ###

### Diapasão

In [ ]:
plt.figure(figsize=(8,1.5))
plt.plot(tempo, diapasao)
plt.xlabel("tempo [s]")
plt.xlim((0,max(tempo)))
plt.xlim((.6,.62));

### SEU CÓDIGO COMEÇA AQUI ### (função que plota a FFT do sinal)
None
### FIM DO CÓDIGO ###

ipd.Audio(diapasao, rate=fs, normalize=True)

### Flauta

In [ ]:
plt.figure(figsize=(8,1.5))
plt.plot(tempo, flauta)
plt.xlabel("tempo (s)")
plt.xlim((0,max(tempo)))
plt.xlim((.6,.62))

### SEU CÓDIGO COMEÇA AQUI ### (função que plota a FFT do sinal)
None
### FIM DO CÓDIGO ###

ipd.Audio(flauta, rate=fs, normalize=True)

### Violoncelo

In [ ]:
plt.figure(figsize=(8,1.5))
plt.plot(tempo, cello)
plt.xlabel("tempo ")
plt.xlim((0,max(tempo)))
plt.xlim((.6,.62))

### SEU CÓDIGO COMEÇA AQUI ### (função que plota a FFT do sinal)
None
### FIM DO CÓDIGO ###

ipd.Audio(cello, rate=fs, normalize=True)

**Saída esperada**:
1. Em cada trecho de código, espera-se o player do audio, o sinal no domínio do tempo e o sinal do domínio da frequência.
---

## Projeto de filtro FIR

Os sinais de áudio estão contaminados com ruído. Sabe-se que os conteúdos espectrais acima de 2,5 kHz podem ser filtrados, pois tratam-se de ruídos.

<mark> **Faça**</mark>: Projete um **filtro FIR** para eliminar ruídos e use o filtro nos sinais.

**ATENÇÃO**: se você chegou até aqui na disciplina, provavelmente já compreendeu os principais passos para *projeto de um filtro* e seu uso. Neste exercício, você tem autonomia para decidir o filtro mais adequado, seus parâmetros de projeto e as melhores formas de apresentar suas características. Não será dada nenhuma instrução específica de como apresentar os resultados, quais gráficos são importantes de serem analisados e nem como apresentar os sinais. Apresente da melhor forma possível, este item fará parte da avaliação.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

___
## Projeto de filtro IIR

Agora, projete um filtro IIR, usando a função `signal.iirdesign`, para realizar a mesma tarefa anterior: filtrar ruídos.

<mark> **Faça**</mark>: Projete um **filtro IIR** usando a função `signal.iirdesign` para eliminar ruídos e use o filtro nos sinais.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

___
## Sinais produzidos pelo filtro FIR vs IIR

<mark> **Faça**</mark>: Usando a função `signal.lfilter`, filtre e compare os sinais produzidos pelo filtro FIR e pelo filtro IIR.

***Como devo comparar?*** A decisão de como comparar os sinais também faz parte da avaliação. Lembre-se das principais diferenças construtivas entre filtros FIR e filtros IIR para determinar qual a melhor forma (gráfica) para ressaltar a diferença entre os sinais produzidos.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### 
None
### FIM DO CÓDIGO ###

___

# Concluões

**1 - Quais as principais diferenças entre o atraso de grupo dos filtros FIR e IIR? Justifique mencionando explicitamente um gráfico produzido.**

**Resposta:**

`[escreva aqui sua resposta]`

**2 - Qual filtro teve maior ordem, FIR ou IIR?**

**Resposta:**

`[escreva aqui sua resposta]`

______________

# Desafio (opcional)

1. Compare a distorção na forma de onda provocada pelo filtro IIR em comparação com o filtro FIR.
1. Compare a aplicação da função "lfilter" com a função "filtfilt" e explique a diferença entre elas.
1. Faça projeto de um filtro Bessel e mostre suas principais características em comparação com os demais.
1. Compare a FFT de um sinal com a FFT do sinal "janelado", usando diferentes janelas. Quais efeitos observados?

<p><a href="https://colab.research.google.com/github/ifmg-betim/2022.2_AUT.040_ProcSin/blob/main/Ex/Ex08_FiltrosDigitais_SEU_NOME.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>